In [8]:
# 📅 2025/06/06 - MobileNetV2 + Adam 튜닝 (RGB Input, ImageNet weights)

import os
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# 데이터 경로 (RGB 이미지가 class 폴더 안에 있음)
data_dir = r"C:\Users\ghwns\HJ_git\CV-Projects\real-time-daily-activity-recognizer\images"

# ImageDataGenerator 설정 (ImageNet 전처리 + 검증 분할)
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=128,
    subset='validation',
    shuffle=False
)

# MobileNetV2 기반 전이학습 모델 정의
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # 최초엔 freeze

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

# 튜닝된 Adam Optimizer
optimizer = Adam(learning_rate=0.0003, beta_1=0.85, beta_2=0.995)

# 콜백 설정
checkpoint_path = "mobilenetv2_adamtune_best.keras"
checkpointer = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, verbose=1)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# 모델 컴파일
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 학습 시작
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[checkpointer, lr_reducer, early_stopper],
    verbose=2
)

# 학습 곡선 저장
fig_path = r"C:\Users\ghwns\HJ_git\CV-Projects\real-time-daily-activity-recognizer\figures\exp_20250606_rgb_transfer_mobilenet_adamtune.png"
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Training / Validation Accuracy (MobileNetV2 + Adam Tuned)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(fig_path)
plt.close()

Found 3001 images belonging to 8 classes.
Found 743 images belonging to 8 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20

Epoch 1: val_loss improved from inf to 2.09551, saving model to mobilenetv2_adamtune_best.keras
24/24 - 29s - 1s/step - accuracy: 0.1529 - loss: 2.4313 - val_accuracy: 0.2275 - val_loss: 2.0955 - learning_rate: 3.0000e-04
Epoch 2/20

Epoch 2: val_loss improved from 2.09551 to 1.81135, saving model to mobilenetv2_adamtune_best.keras
24/24 - 40s - 2s/step - accuracy: 0.2612 - loss: 1.9911 - val_accuracy: 0.3405 - val_loss: 1.8113 - learning_rate: 3.0000e-04
Epoch 3/20

Epoch 3: val_loss improved from 1.81135 to 1.63444, saving model to mobilenetv2_adamtune_best.keras
24/24 - 25s - 1s/step - accuracy: 0.3622 - loss: 1.7429 - val_accuracy: 0.4051 - val_loss: 1.6344 - learning_rate: 3.0000e-04
Epoch 4/20

Epoch 4: val_loss improved from 1.63444 to 1.52237, saving model to mobilenetv2_adamtune_best.keras
24/24 - 18s - 745ms/step - accuracy: 0.4299 - 